In [121]:
#import dask.dataframe as dd
#from dask.multiprocessing import get
import itertools
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, roc_curve, mean_squared_error,mean_absolute_error, f1_score
from sklearn.model_selection import KFold, StratifiedKFold,GroupKFold
#import matplotlib.pyplot as plt
#import seaborn as sns
import warnings
from sklearn.preprocessing import LabelEncoder
from utils import *
#from utils2 import *
#from utils3 import *
from datetime import datetime
from datetime import timedelta
#from tqdm import tqdm
#test

warnings.simplefilter(action='ignore', category=FutureWarning)



USE_KFOLD = True

data_path = './input/'
output_path = '../'

In [122]:
# 将能转换成float的转换，不能的填充为nan
def astype(x,t):
    try:
        return t(x)
    except:
        return np.nan

In [123]:
# x为整数返回0，为小数返回1
def have_0(x):
    try:
        r = x.split('.')[1][-1]
        return 0 if r=='0' else 1
    except:
        return 1

In [124]:
str_dict = {'1_total_fee': 'str',
 '2_total_fee': 'str',
 '3_total_fee': 'str',
 '4_total_fee': 'str',
 'pay_num': 'str',
 }

In [125]:
# 跟金额相关的特征
have_0_c = ['1_total_fee','2_total_fee','3_total_fee','4_total_fee','pay_num']

In [126]:
def deal(data):
    for c in have_0_c:
        data['have_0_{}'.format(c)] = data[c].apply(have_0)
        try:
            data[c] = data[c].astype(float)
        except:
            pass
    data['2_total_fee'] = data['2_total_fee'].apply(lambda x: astype(x,float))
    data['3_total_fee'] = data['3_total_fee'].apply(lambda x: astype(x,float))
    data['age'] = data['age'].apply(lambda x: astype(x,int))
    data['gender'] = data['gender'].apply(lambda x: astype(x,int))
    data.loc[data['age']==0,'age'] = np.nan
    data.loc[data['1_total_fee'] < 0, '1_total_fee'] = np.nan
    data.loc[data['2_total_fee'] < 0, '2_total_fee'] = np.nan
    data.loc[data['3_total_fee'] < 0, '3_total_fee'] = np.nan
    data.loc[data['4_total_fee'] < 0, '4_total_fee'] = np.nan
    for c in [
    '1_total_fee', '2_total_fee', '3_total_fee', '4_total_fee',
    'month_traffic', 'last_month_traffic', 'local_trafffic_month',
    'local_caller_time', 'service1_caller_time', 'service2_caller_time',
    'many_over_bill', 'contract_type', 'contract_time', 'pay_num', ]:
        data[c] = data[c].round(4)
    # 添加一列，有重复的数据，该列为True，无重复数据，该列为False
    data['is_duplicated'] =data.duplicated(subset=['1_total_fee','2_total_fee','3_total_fee', 
 'month_traffic','pay_times','last_month_traffic','service2_caller_time','age'],keep=False)
    return data

In [127]:
train = pd.read_csv(data_path + 'train.csv',dtype=str_dict)

In [128]:
train = train[train['current_service'] != 999999]

In [129]:
train = deal(train)

In [130]:
test = pd.read_csv(data_path + 'test.csv',dtype=str_dict)
test = deal(test)

In [131]:
train_old = pd.read_csv('./input/train_old.csv',dtype=str_dict)[:]
train_old = deal(train_old)

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [132]:
def get_magic_feature(df, outname):
    """
    It is the magic beer and niaobu, try it and enjoy!
    """
    # 每一个样本total_fee 不一样的个数
    df['fea_unum'] = df[['1_total_fee','2_total_fee','3_total_fee', '4_total_fee']].nunique(axis=1)
    df.drop_duplicates(subset =['1_total_fee','2_total_fee','3_total_fee', '4_total_fee'],inplace=True)
    # 保留不一样个数大于两个的
    df = df[df.fea_unum>2]
    # 写出所有的月份组合
    for month1_month2 in [
        [1,3],
        [1,4],
        [2,1],
        [2,3],
        [2,4],
        [3,1],
        [3,2],
        [3,4],
        [4,1],
        [4,2],
        [4,3],
    ]:
        month1, month2 = str(month1_month2[0]), str(month1_month2[1])
        mstr = '_total_fee'
        # 每一个月份组合下，两个total_fee的频数统计
        tmp = df.groupby([month1 + mstr, month2 + mstr]).size().reset_index()
        tmp.columns =['first','second','{}_total_fee_{}_total_fee'.format(month1,month2)]
        # 将频数统计合并
        if month1_month2 == [1,3]:
            result_df = tmp
        else:
            result_df = result_df.merge(tmp, on = ['first','second'], how = 'outer')

    tmpall = result_df
    tmpall = tmpall[tmpall.second!=0]
    tmpall['count'] =  tmpall.iloc[:,2:].sum(axis=1)
    tmpall = tmpall.merge(tmpall.groupby('second',as_index=False)['count'].agg({'sum':'sum'}),on='second',how='left')
    tmpall['rate'] = tmpall['count'] / tmpall['sum']
    tmpall = tmpall.sort_values(['first','rate'],ascending=False)
    tmpall =  tmpall [tmpall['count']>10]
    tmpall = tmpall.sort_values(['first','count'],ascending=False)
    tmp_res = tmpall.drop_duplicates('first',keep='first')
    tmp_res[tmp_res['count']>10].to_csv(outname, columns = ['first','count'],index = False)

In [133]:
train = train.append(test).reset_index(drop = True)

In [134]:
get_magic_feature(train, 'Magic_Feature_Exclude_Old.csv')

D:\anaconda\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [135]:
train.drop_duplicates(subset = ['1_total_fee','2_total_fee','3_total_fee',
 'month_traffic','pay_times','last_month_traffic','service2_caller_time','age'],inplace=True)

In [136]:
train = train.append(train_old).reset_index(drop = True)
train.drop_duplicates(subset = ['1_total_fee','2_total_fee','3_total_fee',
 'month_traffic','pay_times','last_month_traffic','service2_caller_time','age'],inplace=True)
get_magic_feature(train, 'Magic_Feature_Include_Old.csv')

D:\anaconda\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [137]:
train.drop_duplicates(subset = ['1_total_fee','2_total_fee','3_total_fee',
 'month_traffic','pay_times','last_month_traffic','service2_caller_time','age'],inplace=True)
get_magic_feature(train, 'Magic_Feature_Include_Old.csv')

D:\anaconda\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
